# PA KELOMPOK 1 KECERDASAN BUATAN

### NAMA ANGGOTA KELOMPOK : 
### - Hadie Pratama Tulili
### - Rafi Izdihar
### - Muhammad Firdaus

# 1.Tujuan

1. Untuk mengaplikasikan Convolutional Neural Networks (CNN) untuk memprediksi tipe kue khas indonesia.

2. Untuk mengklasifikasikan setiap gambar kue ke salah satu dari 8 kategori : 
    **kue_risoles, kue_lumpur, kue_kastengel, kue_dadar_gulung, kue_putri_salju, kue_klepon,kue_serabi, kue_lapis**
    
3. Jumlah data training yang digunakan sebesar ??? sampel, jumlah data testing sebesar ??? sampel serta jumlah data validation sebesar ??? sampel.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras_preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# directory list
train_path = './cakes-dataset/train/'
test_path = './cakes-dataset/test/'
val_path = './cakes-dataset/validation/'

In [75]:
#Training
train = tf.keras.utils.image_dataset_from_directory(train_path,
                                                            batch_size=32,                                                        
                                                            image_size=(150, 150),
                                                            labels='inferred',
                                                            label_mode='categorical',
                                                            shuffle=True)

#Validation
validation = tf.keras.utils.image_dataset_from_directory(val_path,
                                                            batch_size=32,
                                                            image_size=(150, 150),
                                                            labels='inferred',
                                                            label_mode='categorical',
                                                            shuffle=True)


#Testing
test = tf.keras.utils.image_dataset_from_directory(test_path,
                                                            batch_size=32,
                                                            image_size=(150, 150),
                                                            labels='inferred',
                                                            label_mode='categorical',
                                                            shuffle=True)                                                            

Found 1642 files belonging to 4 classes.
Found 392 files belonging to 4 classes.
Found 512 files belonging to 4 classes.


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                rotation_range = 0.45,
                                zoom_range = 0.2,
                                horizontal_flip = True,
                                fill_mode = 'nearest')

training_set = train_datagen.flow_from_directory(train_path,
                                                batch_size=64,
                                                target_size=(150,150),
                                                shuffle=True,
                                                color_mode = "rgb",
                                                class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255,
                                rotation_range = 0.45,
                                zoom_range = 0.2,
                                horizontal_flip = True,
                                fill_mode = 'nearest')

validation_set = val_datagen.flow_from_directory(val_path,
                                                batch_size=64,
                                                target_size=(150,150),
                                                shuffle=True,
                                                color_mode = "rgb",
                                                class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255,
                                rotation_range = 0.45,
                                zoom_range = 0.2,
                                horizontal_flip = True,
                                fill_mode = 'nearest')

test_set = test_datagen.flow_from_directory(test_path,
                                                batch_size=64,
                                                target_size=(150,150),
                                                shuffle=True,
                                                color_mode = "rgb",
                                                class_mode='categorical')

In [ ]:
# class names
class_names = train.class_names
class_names

# Visualisasi

In [ ]:
# data visualization
plt.figure(figsize=(10, 10))
for images, labels in train.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[np.argmax(labels[i])])
    plt.axis("off")

In [ ]:
train_temp = train.unbatch()
labels_train = []
for x, y in train_temp:
    index = np.argmax(y)
    labels_train.append(class_names[index])

In [ ]:
val_temp = validation.unbatch()
labels_val = []
for x, y in val_temp:
    index = np.argmax(y)
    labels_val.append(class_names[index])

In [ ]:
plt.figure(figsize=(10, 8))

sns.set_theme()

sns.countplot(x=labels_train)
plt.xlabel('Kue Indonesia')
plt.ylabel('Jumlah')
plt.title('Beberapa Nama Kelas Yang Ada di Train Set')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))

sns.set_theme()


sns.countplot(x=labels_val)
plt.xlabel('Kue Indonesia')
plt.ylabel('Jumlah')
plt.title('Beberapa Nama Kelas Yang Ada di Val Set')
plt.show()

In [76]:
#Normalisasi
def normalize_img(image, label):
  return tf.cast(image, tf.float32)/255 , label

#Tambah tingkat keterangan
def random_brightness(image, label):
  return tf.image.random_brightness(image,max_delta=0.7), label

In [77]:
train = train.map(normalize_img)
validation = validation.map(normalize_img)
test = test.map(normalize_img)

In [78]:
model = tf.keras.Sequential([

tf.keras.layers.Conv2D(64, (3,3), activation='relu',input_shape=[150, 150, 3]),
tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
tf.keras.layers.Dropout(0.7),

tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
tf.keras.layers.Dropout(0.6),

tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
tf.keras.layers.Dropout(0.5),

tf.keras.layers.Conv2D(8, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(pool_size = (2,2)),

tf.keras.layers.Flatten(),
tf.keras.layers.Dense(4, activation='softmax')
])

In [79]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 74, 74, 64)       0         
 g2D)                                                            
                                                                 
 dropout_15 (Dropout)        (None, 74, 74, 64)        0         
                                                                 
 conv2d_21 (Conv2D)          (None, 72, 72, 32)        18464     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                            
                                                                 
 dropout_16 (Dropout)        (None, 36, 36, 32)       

In [80]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics=['accuracy'])

In [81]:
earlyStopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=10,
)

In [ ]:
hasil1 = model.fit(training_set, epochs=30,validation_data=validation_set,verbose=1,callbacks=[earlyStopping])

In [82]:
hasil = model.fit(train, epochs=30,validation_data=validation,verbose=1,callbacks=[earlyStopping])

Epoch 1/30
52/52 [==============================] - 73s 1s/step - loss: 1.4168 - accuracy: 0.2722 - val_loss: 1.3866 - val_accuracy: 0.2551
Epoch 2/30
52/52 [==============================] - 68s 1s/step - loss: 1.3592 - accuracy: 0.3045 - val_loss: 1.3843 - val_accuracy: 0.2577
Epoch 3/30
52/52 [==============================] - 70s 1s/step - loss: 1.3251 - accuracy: 0.3508 - val_loss: 1.3806 - val_accuracy: 0.2628
Epoch 4/30
52/52 [==============================] - 70s 1s/step - loss: 1.2007 - accuracy: 0.4501 - val_loss: 1.2724 - val_accuracy: 0.4337
Epoch 5/30
52/52 [==============================] - 68s 1s/step - loss: 1.1349 - accuracy: 0.4775 - val_loss: 1.3008 - val_accuracy: 0.3163
Epoch 6/30
52/52 [==============================] - 68s 1s/step - loss: 1.0137 - accuracy: 0.5566 - val_loss: 1.1449 - val_accuracy: 0.5102
Epoch 7/30
52/52 [==============================] - 66s 1s/step - loss: 0.8829 - accuracy: 0.6346 - val_loss: 1.0146 - val_accuracy: 0.5791
Epoch 8/30
52/52 [==

KeyboardInterrupt: 

In [ ]:
test_evaluate = model.evaluate(test_set, verbose=0)
print(f'Test loss     : {test_evaluate[0]}')
print(f'Test accuracy : {test_evaluate[1]}')

In [ ]:
epoch = [i+1 for i in range(30)]


plt.figure(figsize=(25, 10))
plt.suptitle('Train VS Validation')

#Subplot Akurasi
plt.subplot(1,2,1)
sns.lineplot(x=epoch,y = hasil.history['accuracy'],label ='Train Accuracy')
sns.lineplot(x=epoch,y = hasil.history['val_accuracy'],label ='Validation Accuracy')

plt.xticks(epoch)
plt.ylabel('Accuracy(%)')
plt.xlabel('N_Epoch')
plt.title("Train/Validation Accuracy")

#Subplot Loss
plt.subplot(1,2,2)
sns.lineplot(x=epoch,y = hasil.history['loss'],label ='Train Loss')
sns.lineplot(x=epoch,y = hasil.history['val_loss'],label ='Validation Loss')

plt.xticks(epoch)
plt.title("Train/Validation Loss")
plt.xlabel('N_Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
image ,label = next(iter(test_set))


plt.figure(figsize=(15,10))
for i in range(9) :
    TrueLabel = class_names[np.argmax(label[i])]
    plt.subplot(3,3,i+1)
    plt.axis('off')
    y_pred = np.argmax(model.predict(image[i][None,...],verbose=0))
    plt.imshow(tf.squeeze(image[i]))
    plt.title(f'label: {TrueLabel}\npredict : {class_names[y_pred]}')

In [ ]:
#Plot prediction Error
plt.figure(figsize=(15,10))

i = 0
j = 0
while (j < 9):
    TrueLabel = class_names[np.argmax(label[i])]
    plt.subplot(3,3,j+1)
    plt.axis('off')
    y_pred = np.argmax(model.predict(image[i][None,...],verbose=0))

    if (TrueLabel != class_names[y_pred]):
        plt.imshow(tf.squeeze(image[i]))
        plt.title(f'label: {TrueLabel} \npredict : {class_names[y_pred]}')
        j+=1
    i+=1

In [ ]:
test_pred = model.predict(test)
test_pred_labes = np.argmax(test_pred,axis = 1)
test_labels1 = np.concatenate([y for x, y in test], axis=0)
test_labels = [np.argmax(y) for y in test_labels1]

In [ ]:
plt.figure(figsize=(20, 16))
cm = confusion_matrix(test_labels,test_pred_labes)

sns.heatmap(cm,annot=True,fmt='d',xticklabels=class_names,yticklabels=class_names)
plt.title("Confusion Matrix - Test Set")
plt.show()

In [ ]:
#Classification Report
print('Classification Report')
print(classification_report(test_pred_labes, test_labels, target_names=class_names))